In [9]:
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Data Preprocessing

# Reshape the images to the format (num_samples, height, width, num_channels)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Converting the data type to float32 and scale pixel values to the range [0, 1]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# One-hot encoding
y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

In [10]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [11]:
from skimage.transform import resize

# Resize the MNIST images to (224, 224) and replicate the single channel to three channels
x_train = np.array([resize(x, (32, 32, 3)) for x in x_train])
x_val = np.array([resize(x, (32, 32, 3)) for x in x_val])
x_test = np.array([resize(x, (32, 32, 3)) for x in x_test])

In [12]:
# Define ResNet-50 Model
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint,EarlyStopping

# Load the ResNet-50 model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Create a new model by adding custom fully connected layers on top of ResNet-50
model_resnet50 = Sequential()
model_resnet50.add(base_model)
model_resnet50.add(GlobalAveragePooling2D())  # Use Global Average Pooling
model_resnet50.add(Dense(1024, activation='relu'))
model_resnet50.add(Dense(10, activation='softmax')) 
# Compile the ResNet-50 model
model_resnet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model_resnet50.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_4 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_5 (Dense)             (None, 10)                10250     
                                                                 
Total params: 25,696,138
Trainable params: 25,643,018
Non-trainable params: 53,120
_________________________________________________________________


In [14]:
# Training
batch_size = 64
epochs = 10

# Callbacks
resnet_checkpoint = ModelCheckpoint('resnet_model.h5', save_best_only=True)
resnet_early_stop = EarlyStopping(patience=3)

# Training ResNet-50 model
model_resnet50.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                   validation_data=(x_val, y_val), callbacks=[resnet_checkpoint, resnet_early_stop])

# Evaluate the model on the test set
resnet50_test_loss, resnet50_test_accuracy = model_resnet50.evaluate(x_test, y_test)
print(f'ResNet-50 Test Loss: {resnet50_test_loss:.4f}')
print(f'ResNet-50 Test Accuracy: {resnet50_test_accuracy:.4f}')

Epoch 1/10
750/750 [==============================] - 83s 46ms/step - loss: 0.1917 - accuracy: 0.9550 - val_loss: 0.3966 - val_accuracy: 0.8879
Epoch 2/10
750/750 [==============================] - 33s 44ms/step - loss: 0.0688 - accuracy: 0.9830 - val_loss: 0.0695 - val_accuracy: 0.9847
Epoch 3/10
750/750 [==============================] - 32s 43ms/step - loss: 0.3068 - accuracy: 0.9328 - val_loss: 0.1296 - val_accuracy: 0.9621
Epoch 4/10
750/750 [==============================] - 33s 44ms/step - loss: 0.0889 - accuracy: 0.9751 - val_loss: 0.0581 - val_accuracy: 0.9837
Epoch 5/10
750/750 [==============================] - 33s 44ms/step - loss: 0.0648 - accuracy: 0.9822 - val_loss: 0.0540 - val_accuracy: 0.9857
Epoch 6/10
750/750 [==============================] - 33s 44ms/step - loss: 0.0499 - accuracy: 0.9858 - val_loss: 0.0528 - val_accuracy: 0.9865
Epoch 7/10
750/750 [==============================] - 32s 43ms/step - loss: 0.1046 - accuracy: 0.9743 - val_loss: 0.1439 - val_accuracy:

In [15]:
# Define the CNN model
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dropout,Dense
model_cnn = Sequential()

model_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))  # Adjust input shape
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(Flatten())
model_cnn.add(Dropout(0.6))
model_cnn.add(Dense(10, activation='softmax')) 

# Compile the CNN model
model_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
model_cnn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 11, 11, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 128)        0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3200)             

In [17]:
# Training 
batch_size = 64
epochs = 10

# Callbacks
cnn_checkpoint = ModelCheckpoint('cnn_model.h5', save_best_only=True)
cnn_early_stop = EarlyStopping(patience=3)

# Training the CNN model
model_cnn.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
              validation_data=(x_val, y_val), callbacks=[cnn_checkpoint, cnn_early_stop])

# Evaluate the model on the test set
cnn_test_loss, cnn_test_accuracy = model_cnn.evaluate(x_test, y_test)
print(f'CNN Test Loss: {cnn_test_loss:.4f}')
print(f'CNN Test Accuracy: {cnn_test_accuracy:.4f}')

Epoch 1/10
750/750 [==============================] - 6s 5ms/step - loss: 0.2120 - accuracy: 0.9317 - val_loss: 0.0614 - val_accuracy: 0.9802
Epoch 2/10
750/750 [==============================] - 4s 5ms/step - loss: 0.0736 - accuracy: 0.9768 - val_loss: 0.0438 - val_accuracy: 0.9868
Epoch 3/10
750/750 [==============================] - 3s 5ms/step - loss: 0.0565 - accuracy: 0.9822 - val_loss: 0.0347 - val_accuracy: 0.9892
Epoch 4/10
750/750 [==============================] - 4s 5ms/step - loss: 0.0478 - accuracy: 0.9847 - val_loss: 0.0355 - val_accuracy: 0.9887
Epoch 5/10
750/750 [==============================] - 3s 5ms/step - loss: 0.0403 - accuracy: 0.9873 - val_loss: 0.0307 - val_accuracy: 0.9912
Epoch 6/10
750/750 [==============================] - 4s 5ms/step - loss: 0.0347 - accuracy: 0.9890 - val_loss: 0.0352 - val_accuracy: 0.9897
Epoch 7/10
750/750 [==============================] - 4s 5ms/step - loss: 0.0317 - accuracy: 0.9898 - val_loss: 0.0314 - val_accuracy: 0.9907
Epoch 

In [18]:
# ensembling
cnn_predictions = model_cnn.predict(x_test)
resnet50_predictions = model_resnet50.predict(x_test)

# Combine predictions using majority voting
ensemble_predictions = np.argmax(cnn_predictions + resnet50_predictions, axis=1)

ensemble_accuracy = np.mean(ensemble_predictions == np.argmax(y_test, axis=1))
print(f"Ensemble Model Accuracy: {ensemble_accuracy}")

313/313 [==============================] - 4s 9ms/step
Ensemble Model Accuracy: 0.9921
